In [1]:
def clusters_spectral(graph, n_clusters):
    """
    Find communities in a graph using Spectral Clustering.

    Args:
        graph (nx.Graph): The input NetworkX graph.
        n_clusters (int): The number of clusters to identify.

    Returns:
        dict: A dictionary mapping community IDs to lists of node members.
    """
    # Check if the graph has edges
    if graph.number_of_edges() > 0:
        # Convert the graph to an adjacency matrix
        adjacency_matrix = nx.to_numpy_array(graph)
        
        # Perform Spectral Clustering
        sc = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', random_state=42)
        labels = sc.fit_predict(adjacency_matrix)
        
        # Extract communities and their members
        communities = {}
        for node, community_id in zip(graph.nodes(), labels):
            if community_id not in communities:
                communities[community_id] = []
            communities[community_id].append(node)
    else:
        communities = {}
        print("No edges in the graph, no communities found.")

    return communities

In [ ]:
# Initialize storage for results
Components = {}
eegICA = all_data[file_num]['eegICA']  # Preprocessed EEG data
plv_bi = all_data[file_num]['plv_bi']  # Phase Locking Value 
graphs = all_data[file_num]['graphs']  # Graphs for the current file
graphs_bi = all_data[file_num]['graphs_bi']  # Binary graphs for the current file
oke = []  # List to store the estimated number of clusters for each graph

# Process each binary graph to estimate the number of clusters
for G in graphs_bi:
    # Compute the normalized Laplacian matrix
    laplacian = nx.normalized_laplacian_matrix(G).toarray()
    
    # Compute and sort eigenvalues of the Laplacian
    eigenvalues = np.sort(np.linalg.eigvals(laplacian))
    
    # Estimate the number of clusters based on the largest spectral gap
    gaps = np.diff(eigenvalues)
    optimal_k = np.argmax(gaps) + 1
    oke.append(optimal_k)

# Visualize the estimated number of clusters using a moving average
plt.plot(moving_average(oke, 20))

# Perform spectral clustering for each graph using the estimated cluster count
communities_s = [clusters_spectral(graphs_bi[i], oke[i]) for i in range(len(graphs_bi))]

# Save the results for the current file
Components[file_num] = {
    'Communities': communities_s,  # Detected communities for each graph
    'Num_Comp': oke}  # List of estimated number of clusters
